If number of slices are not equal we need to do some processing so that input is consistent for the Neural Network <br>
1. Convert each NIFTI file into a folder of DICOMs <br>
2. Split DICOMs into groups of 64 (try doing for 16, 32, 64, 128 and see results) <br>
3. Convert groups into NIFTI files <br>

<h3> File Structure <br></h3>
&minus; Liver Segmentation <br>
&emsp;&minus; Original <br>
&emsp;&emsp;&minus; training <br>
&emsp;&emsp;&minus; segmentation <br>
&emsp;&minus; DICOM Series <br>
&emsp;&emsp;&minus; training <br>
&emsp;&emsp;&minus; segmentation <br>
&emsp;&minus; DICOM Group <br>
&emsp;&emsp;&minus; training <br>
&emsp;&emsp;&minus; segmentation <br>
&emsp;&minus; NIfTI <br>
&emsp;&emsp;&minus; Images <br>
&emsp;&emsp;&minus; Labels <br>

<h1>1. Convert NIfTI file to DICOM slices</h1>

%pip install nibabel <br>
%pip install pydicom <br>
%pip install numpy <br>


In [29]:
import nibabel
import pydicom
import numpy as np
import os
import shutil
from glob import glob

In [ ]:
def convertNsave(arr, file_dir, index):

    dicom_file = pydicom.dcmread('../images/sampleDICOM.dcm')
    arr = arr.astype('uint16')
    dicom_file.Rows = arr.shape[0]
    dicom_file.Columns = arr.shape[1]
    dicom_file.PhotometricInterpretation = "MONOCHROME2"
    dicom_file.SamplesPerPixel = 1
    dicom_file.BitsStored = 16
    dicom_file.BitsAllocated = 16
    dicom_file.HighBit = 15
    dicom_file.PixelRepresentation = 1
    dicom_file.PixelData = arr.tobytes()
    dicom_file.save_as(os.path.join(file_dir, f'slice{index}.dcm'))

In [ ]:
def nifti2dicom(nifti_dir, out_dir):

    files = os.listdir(nifti_dir)
    for file in glob(nifti_dir + '/*'):
        filename = os.path.splitext(os.path.basename(file))[0]
        out_path = os.path.join(out_dir, filename)
        os.mkdir(out_path)

        nifti_file = nibabel.load(file)
        nifti_array = nifti_file.get_fdata()
        number_slices = nifti_array.shape[2]
        
        for slice in range(number_slices):
            convertNsave(nifti_array[:,:,slice], out_path, slice)

Move all the training files from separate volumes into one folder

In [ ]:
in_path = 'D:/Datasets/LiverSegmentation/Original/training'

for volume in os.listdir(in_path):
    path = os.path.join(in_path, volume)
    for file in os.listdir(path):
        shutil.move(os.path.join(path, file), in_path)
    shutil.rmtree(path)

In [ ]:
nifti2dicom('D:/Datasets/LiverSegmentation/Original/training', 'D:/Datasets/LiverSegmentation/DICOM/training')
nifti2dicom('D:/Datasets/LiverSegmentation/Original/segmentations', 'D:/Datasets/LiverSegmentation/DICOM/segmentations')

Delete the original data to save space on disk

In [ ]:
shutil.rmtree('D:/Datasets/LiverSegmentation/Original')

<h1>2. Creating group of DICOM slices</h1>

%pip install glob2 <br>
%pip install pytest-shutil

In [ ]:
from glob import glob
import shutil
import os

In [ ]:
def createGroups(in_path, out_path, group_size):
    for patient in glob(in_path + '/*'):
        patient_name = os.path.basename(os.path.normpath(patient))
        num_groups = int(len(glob(patient + '/*')) / group_size)
        
        for i in range(num_groups):
            output_path = os.path.join(out_path, patient_name + '_' + str(i))
            os.mkdir(output_path)
            
            for index, file in enumerate(glob(patient + '/*')):
                if index == group_size:
                    break
                shutil.move(file, output_path)

In [ ]:
group_size = 64

createGroups('D:/Datasets/LiverSegmentation/DICOM/training', 'D:/Datasets/LiverSegmentation/Groups/training', group_size)
createGroups('D:/Datasets/LiverSegmentation/DICOM/segmentations','D:/Datasets/LiverSegmentation/Groups/segmentations', group_size)

Delete the leftover DICOM files that are not in any group to save space on disk

In [ ]:
shutil.rmtree('D:/Datasets/LiverSegmentation/DICOM')

Groups will be created for the lower multiple of group_size. The remaining files will be discarded (total % group_size), however this should not affect final results (Can try other batch sizes and see difference)

<h1>3. Converting DICOM Group to NIfTI</h1>

%pip install dicom2nifti

In [ ]:
import dicom2nifti

In [ ]:
def convertToNifti(in_path, out_path):
    list = glob(in_path + '/*')
    for patient in list:
        patient_name = os.path.basename(os.path.normpath(patient))
        dicom2nifti.dicom_series_to_nifti(patient, os.path.join(out_path, patient_name + '.nii.gz'))

In [ ]:
convertToNifti('D:/Datasets/LiverSegmentation/Groups/training', 'D:/Datasets/LiverSegmentation/NIfTI/images')
convertToNifti('D:/Datasets/LiverSegmentation/Groups/segmentations', 'D:/Datasets/LiverSegmentation/NIfTI/labels')

Delete the DICOM groups to save space on disk

In [ ]:
shutil.rmtree('D:/Datasets/LiverSegmentation/Groups')

<h1> Finding Empty Labels </h1> <br>
Some NIfTI files may not have any segmentation in them, that is, they contain only background pixels, which will cause a problem to the learning algorithm. To remove such files, we check every label NIfTI file. If it contains just one label, that is 0 (background), we will delete that label and its corresponding image file

In [ ]:
path_labels = 'D:/Datasets/LiverSegmentation/NIfTI/labels/*'
path_images = 'D:/Datasets/LiverSegmentation/NIfTI/images'
list_labels = glob(path_labels)

for patient in list_labels:
    nifti_file = nibabel.load(patient)
    nifti_array = nifti_file.get_fdata()
    num_labels = len(np.unique(nifti_array))
    if num_labels == 1:
        group_name = os.path.basename(patient)
        image_path = os.path.join(path_images, group_name)
        os.remove(patient)
        os.remove(image_path)